# Importing all the important libraries

In [1]:
import numpy as np
import h5py as h5

import torch
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

import matplotlib.pyplot as plt

import glob
import os
os.chdir("..")

import pdb
import h5py
from PIL import Image as im

## Loading and processing train and test datasets
### Using Kalantari dataset

In [2]:
#test1_dir = "Documents\\Datasets\\Kalantari\\TestSet\\PAPER\\"
#test2_dir = "Documents\\Datasets\\Kalantari\\TestSet\\EXTRA\\"
#train1_dir = "Documents\\Datasets\\Srinivasan\\"
#train2_dir = "Documents\\Datasets\\Kalantari\\TrainingSet\\"
#os.getcwd()
#model_dir = "Documents\\Datasets\\Kalantari\\Models_300k\\MYTR_MAE_210K.tar"
#model_dir = "Documents\\Datasets\\Kalantari\\Models_300k\\MYTR_MAE_WD1_51K.tar"
#model_dir = "Documents\\Datasets\\Kalantari\\Models_300k\\OAVS-Copy5_400K.tar"

if os.name == 'posix':
    model_dir = "Models/OAVS_grad1_flowers.tar.waffine"
    #imgDir = "../../../../Documents/Datasets/Kalantari/TestSet/PAPER/"
    #Network_Name = 'Networks.UDDE_000'
    
else:
    model_dir = "Models\\OAVS_grad1_flowers.tar.wawob"
    #imgDir = "..\\..\\..\\..\\Documents\\Datasets\\Kalantari\\TestSet\\PAPER"
    #imgDir = "..\\..\\..\\..\\Documents\\Datasets\\Srinivasan\\Flowers_8bit\\TestSet"
    imgDir = "..\\..\\..\\..\\Documents\\Datasets\\Kalantari\\TestSet\\EXTRA"
    Network_Name = 'Networks.OAVS_grad1_flowers'

In [3]:
gamma_val = 0.4
minibatch_size = 1
lfsize = [372, 540, 7, 7] #dimensions of Lytro light fields

In [4]:
def processLF(lf):
    
    # Crop image
    lf = lf[:3,:lfsize[0]*14,:lfsize[1]*14]
    # 2D lenslet grid to 4D
    lf = lf.view(3, lfsize[0], 14, lfsize[1], 14).permute(1, 3, 2, 4, 0)
    # Pick the central perspectives only
    lf = lf[:, :, (14//2)-(lfsize[2]//2):(14//2)+(lfsize[2]//2) + 1, (14//2)-(lfsize[3]//2):(14//2)+(lfsize[3]//2) + 1, :]
    # Gamma correction
    lf = torch.pow(lf, gamma_val)
    # Normalize LF (-1 to 1)
    lf = (lf * 2.0) - 1.0
    
    return lf

def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if torch.cuda.is_available():
        return x.cuda()
    return x

def get_numpy(x):
    """ Get numpy array for both cuda and not. """
    if torch.cuda.is_available():
        return x.cpu().data.numpy()
    return x.data.numpy()

import math

def psnr_1(img1, img2):
    mse = np.mean( ((img1 - img2)/2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 1.0
    return 10 * math.log10(PIXEL_MAX / mse)

In [5]:
trans = transforms.ToPILImage()
trans1 = transforms.ToTensor()
p = np.ndarray([1])
q = np.ndarray([1])

In [6]:
import importlib
import sys
sys.path.insert(1, '\\Networks')
network_module = importlib.import_module(Network_Name)
Net = network_module.Net

net = Net((lfsize[0], lfsize[1]), minibatch_size, lfsize, batchAffine=True)
if torch.cuda.is_available():
    print('##converting network to cuda-enabled')
    net.cuda()
print(net)

try:
    checkpoint = torch.load(model_dir)
    net.load_state_dict(checkpoint['model'].state_dict())    
    print('Model successfully loaded.')
    
except:
    print('No model.')

##converting network to cuda-enabled
Net(
  (f_conv0): Conv2d(5, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (f_conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (f_conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (f_conv3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (f_conv4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (f_pool0): AvgPool2d(kernel_size=16, stride=16, padding=0)
  (f_pool1): AvgPool2d(kernel_size=8, stride=8, padding=0)
  (f_conv5): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (f_bn0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (f_bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (f_bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (f_bn3): BatchNor

In [7]:
# To delete
#result = im.fromarray((get_numpy(corn[0,:3].permute(1,2,0)+1)/2 * 255).astype(np.uint8));result.save('corner1_f.png');
#result = im.fromarray((get_numpy(corn[0,3:6].permute(1,2,0)+1)/2 * 255).astype(np.uint8));result.save('corner2_f.png');
#result = im.fromarray((get_numpy(corn[0,6:9].permute(1,2,0)+1)/2 * 255).astype(np.uint8));result.save('corner3_f.png');
#result = im.fromarray((get_numpy(corn[0,9:].permute(1,2,0)+1)/2 * 255).astype(np.uint8));result.save('corner4_f.png');

In [8]:
#with torch.no_grad():
#    O_view = net(get_variable(corn), get_variable(torch.from_numpy(p)), get_variable(torch.from_numpy(q)))
#    net_out = get_numpy(O_view)
#    Y = get_numpy(pers)
#
#    ps = psnr_1(np.squeeze(net_out), np.squeeze(Y))
#    print(ps)

In [7]:
import cv2

files = [file for file in os.listdir(imgDir) if file.endswith(".png")]

ps = np.ndarray((len(files),7,7))

for fi in range(len(files)):

    file = files[fi]
    
    print("Current file {}: {}" .format(fi,file))
    
    img = cv2.imread(os.path.join(imgDir, file))
                
    img = processLF(trans1(img))

    for i in range(7):
        for j in range(7):
            pers = img[:, :, i, j, :].squeeze()
            corn = img[:, :, [0, -1, 0, -1], [0, 0, -1, -1], :].squeeze()

            p[0] = (i - lfsize[2]//2)/(lfsize[2]//2)
            q[0] = (j - lfsize[3]//2)/(lfsize[3]//2)

            corn = corn.permute(2,3,0,1).reshape(12,pers.shape[0],pers.shape[1])[None,:]
            pers = pers[None,:].permute(0,3,1,2)

            with torch.no_grad():
                O_view = net(get_variable(corn), get_variable(torch.from_numpy(p)), get_variable(torch.from_numpy(q)))

                net_out = get_numpy(O_view)
                Y = get_numpy(pers)

                ps[fi,i,j] = psnr_1(np.squeeze(net_out), np.squeeze(Y))      

                del O_view            
                torch.cuda.empty_cache()
    print("Current PSNR: {}" .format(ps[fi].mean()))
                
    

Current file 0: Cars.png


C:\Users\mummu\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\nn\functional.py:2404: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


Current PSNR: 29.838377937562775
Current file 1: Flower1.png
Current PSNR: 31.267416963265916
Current file 2: Flower2.png
Current PSNR: 31.336501083164425
Current file 3: IMG_1085_eslf.png
Current PSNR: 36.95246996390464
Current file 4: IMG_1086_eslf.png
Current PSNR: 35.49888573774054
Current file 5: IMG_1184_eslf.png
Current PSNR: 35.04667170018851
Current file 6: IMG_1187_eslf.png
Current PSNR: 32.815261879454916
Current file 7: IMG_1306_eslf.png
Current PSNR: 32.77327449132135
Current file 8: IMG_1312_eslf.png
Current PSNR: 38.63503691703915
Current file 9: IMG_1316_eslf.png
Current PSNR: 21.0835681747482
Current file 10: IMG_1317_eslf.png
Current PSNR: 27.60736423802953
Current file 11: IMG_1320_eslf.png
Current PSNR: 32.38700398224729
Current file 12: IMG_1321_eslf.png
Current PSNR: 35.47449045535484
Current file 13: IMG_1324_eslf.png
Current PSNR: 38.013584655701436
Current file 14: IMG_1325_eslf.png
Current PSNR: 31.672742002063252
Current file 15: IMG_1327_eslf.png
Current PSN

In [8]:
np.set_printoptions(precision=2)
print(ps)

#for index, val in enumerate(list)

[[[29.82 30.31 30.47 30.53 30.62 30.56 30.01]
  [30.77 31.34 31.42 31.48 31.68 31.77 31.12]
  [31.15 31.77 31.98 32.09 32.33 32.22 31.53]
  [30.86 31.44 32.02 32.26 32.34 32.21 31.54]
  [30.78 31.81 32.23 32.41 32.41 32.34 31.55]
  [30.75 31.63 31.97 31.89 31.84 31.74 30.91]
  [30.04 30.91 31.31 31.26 31.2  30.76 29.88]]

 [[31.43 32.03 31.99 31.94 31.93 31.72 31.35]
  [31.83 32.4  32.3  32.14 32.15 31.96 31.57]
  [32.16 32.53 32.43 32.28 32.29 32.12 31.65]
  [32.1  32.4  32.38 32.26 32.34 32.07 31.65]
  [31.96 32.25 32.19 32.16 32.22 32.05 31.69]
  [31.46 31.9  31.9  31.86 31.91 31.68 31.22]
  [30.81 31.27 31.41 31.42 31.49 31.04 30.52]]

 [[30.56 30.91 30.99 30.8  30.69 30.41 29.44]
  [31.28 31.95 32.05 31.92 31.85 31.29 30.17]
  [31.72 32.26 32.41 32.25 32.24 31.68 30.31]
  [31.22 31.95 32.16 32.06 32.09 31.51 30.31]
  [30.92 31.57 31.76 31.72 31.7  31.26 30.2 ]
  [29.85 30.52 30.7  30.75 30.79 30.18 29.29]
  [29.02 29.58 29.91 29.91 29.94 29.15 28.28]]

 [[27.6  27.96 27.65 27.37 2

In [14]:
#for _, fi in enumerate(files):
ps.reshape(len(files),-1)[:,[i for i in range(49) if (i-np.array((0,6,42,48))).all()]].mean()

33.902742582728166

In [ ]:
img = im.open(imgDir + 'Cars.png')

In [8]:
ps.reshape(-1,len(files))[[i for i in range(49) if (i-np.array((0,6,42,48))).all()]].mean()

31.291147203083646

In [10]:
ps.reshape(-1,len(files)).mean()

36.62863017513677

In [13]:
ps.reshape(len(files),-1).shape

(5, 49)

In [14]:
import cv2


files = [file for file in os.listdir(imgDir) if file.endswith(".png")]

ps = np.ndarray((len(files),7,7))

fi = files.index("Seahorse.png")

file = files[fi]

print("Current file {}: {}" .format(fi,file))

img = cv2.imread(os.path.join(imgDir, file))

img = processLF(trans1(img))

i = 0
j = 0

pers = img[:, :, i, j, :].squeeze()
corn = img[:, :, [0, -1, 0, -1], [0, 0, -1, -1], :].squeeze()

#pdb.set_trace()

p[0] = (i - lfsize[2]//2)/(lfsize[2]//2)
q[0] = (j - lfsize[3]//2)/(lfsize[3]//2)

corn = corn.permute(2,3,0,1).reshape(12,pers.shape[0],pers.shape[1])[None,:]
pers = pers[None,:].permute(0,3,1,2)

with torch.no_grad():
    O_view = net(get_variable(corn), get_variable(torch.from_numpy(p)), get_variable(torch.from_numpy(q)))

    net_out = get_numpy(O_view)
    Y = get_numpy(pers)

    ps[fi,i,j] = psnr_1(np.squeeze(net_out), np.squeeze(Y))      

    del O_view            
    torch.cuda.empty_cache()

Current file 4: Seahorse.png
> c:\users\mummu\documents\github\deep-learning-projects\oavs-navarro\networks\oavs_grad1_flowers.py(219)forward()
-> return I
(Pdb) result = im.fromarray((get_numpy(d[0,0]+4)/8 * 255).astype(np.uint8));result.save('depth1.png');
*** NameError: name 'im' is not defined
(Pdb) from PIL import Image as im
(Pdb) result = im.fromarray((get_numpy(d[0,0]+4)/8 * 255).astype(np.uint8));result.save('depth1.png');
(Pdb) result = im.fromarray((get_numpy(x_00[0].permute(1,2,0)+1)/2 * 255).astype(np.uint8));result.save('image1.png');
(Pdb) q


BdbQuit: 